In [ ]:
# Run this cell first

!git clone https://github.com/Mlodko/reed-solomon.git
%cd reed-solomon

In [ ]:
from rs import RSCoder, binary_format

n = 255 # @param {type:"slider", min:0, max:255, step:1}
k = 223 # @param {type:"slider", min:0, max:255, step:1}
input = "" # @param {type:"string", placeholder:"Enter a message to be encoded"}

coder = RSCoder(255,223)
print(f"Unencoded message:\n\tBinary representation: \n{binary_format(input)}")
print("\n=========================================\n")
encoded = coder.encode(input)
print(f"Encoded message:\n\tBinary representation: \n{binary_format(encoded)}")
print("\n=========================================\n")
print(f"Decoded message:\n\tBinary representation: \n{binary_format(coder.decode(encoded))}")